## Modelo Machine learning para Megaline

## 1. Contenido

* [1 Contenido](#content)
* [2 Introducción](#intro)
* [3 Inicialización](#inic)
    * [3.1 Cargar Librerias](#library)
    * [3.2 Cargar Datos](#datos)
* [4 Exploración de los datos](#exp)       
* [5 Segmentación de los datos](#segmentacion) 
* [6 Mejoramiento de los modelos](#mejoramiento)
    * [6.1 Árbol de desición](#arbol)
    * [6.2 Bosque aleatorio](#bosque)
    * [6.3 Regresión logística](#regresion)    
* [7 Calidad y elección del modelo](#model)
* [8 Conclusión general](#end)

## 2. Introducción

En el presente proyecto desarrollaremos un modelo que analice el comportamiento de los clientes de la empresa Megaline y recomiende uno de los nuevos planes Smart o Ultra de la empresa. Comenzaremos segmentando los datos, analizaremos distintos tipos de modelos para posteriormente elegir el que mejor resultado tenga y ponerlo a prueba con la base de datos otorgada por Megaline.

## 3. Inicialización

### 3.1 Cargar librerias

Se procede a cargar las librerías que se utilizaran en el proyecto.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### 3.2 Cargar datos

Se procede a cargar los datos.

In [2]:
plan = pd.read_csv('datasets/users_behavior.csv')

## 4. Exploración de los datos

A continuación, le daremos un vistazo a la base de datos entregada por Megaline, como ya se realizó un análisis de las mismas bases de datos en un proyecto anterior se procederá con una exploración rápida para ver que todo se encuentre en orden.

In [3]:
plan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
plan.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
for column in plan:
    print(f'{column} {len(plan[plan[column]>=0])}')

calls 3214
minutes 3214
messages 3214
mb_used 3214
is_ultra 3214


In [6]:
len(plan['is_ultra'].unique())

2

Como podemos observar no hay datos ausentes en el dataframe, no hay datos extraños como valores negativos en las columnas y la columna "is_ultra" solo tiene dos tipos distintos de valores, 0 y 1, como debería ser al ser una columna con datos binarios. Con respecto al dataframe en sí mismo podemos apreciar que la columna "is_ultra" será nuestra columna "objetivo" que intentara predecir nuestro modelo y el resto de las columnas serán las "características" que ayudaran al modelo a realizar las predicciones. Es de importancia resaltar que nuestra columna objetivo es de tipo categórica ya que consiste en un valor binario que indica si el plan es de un tipo u otro, por lo que en base a esta característica serán los algoritmos de aprendizaje que se utilizarán más adelante.

## 5. Segmentación de los datos

Comenzaremos dividiendo el dataframe en dos, uno con las columnas "características" y otro con la columna "objetivo".

In [7]:
features = plan.drop('is_ultra', axis=1)
target = plan['is_ultra']

Ahora procederemos a realizar la segmentación de los datos en tres conjuntos, uno de entrenamiento, uno de validación y uno de prueba. Para llevarlo a cabo utilizaremos la función "train_test_split" la cual separa nuestro dataframe en dos, como necesitamos tres conjuntos realizaremos este proceso dos veces para conseguir los conjuntos anteriormente mencionados en una proporción de 3:1:1.

In [8]:
features_train1, features_valid, target_train1, target_valid = train_test_split(
    features, target, test_size=0.2, random_state=12345)

In [9]:
features_train, features_test, target_train, target_test = train_test_split(
    features_train1, target_train1, test_size=0.25, random_state=12345)

In [10]:
print(f'features_train {len(features_train)/len(features)}')
print(f'features_train {len(features_valid)/len(features)}')
print(f'features_train {len(features_test)/len(features)}')

features_train 0.5998755444928439
features_train 0.2000622277535781
features_train 0.2000622277535781


Podemos observar que hemos conseguido los tres conjuntos de dataframes en las proporciones requeridas.

## 6. Mejoramiento de los modelos

A continuación, evaluaremos independientemente distintos tipos de algoritmos de aprendizaje ajustando sus hiperpárametros para mejorar el modelo y obtener la mayor exactitud posible.

### 6.1 Árbol de decisión

In [11]:
for depth in range(1,8):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    print(f'max_depth = {depth} : {result}')

max_depth = 1 : 0.7480559875583204
max_depth = 2 : 0.7838258164852255
max_depth = 3 : 0.7869362363919129
max_depth = 4 : 0.7869362363919129
max_depth = 5 : 0.7884914463452566
max_depth = 6 : 0.7791601866251944
max_depth = 7 : 0.7884914463452566


Comenzamos analizando distintos valores de "profundidad máxima" para el árbol de decisión, con las pruebas realizadas el valor más exacto para las predicciones realizadas es con una profundidad de 3 y 4, por lo que nos decantaremos por el primer valor para reducir tiempos de carga asociados a mayor profundidad en el modelo.

In [12]:
for leaf in range(1,8):
    model = DecisionTreeClassifier(random_state=12345, max_depth=5, min_samples_leaf=leaf)
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    print(f'min_samples_leaf = {leaf} : {result}')

min_samples_leaf = 1 : 0.7884914463452566
min_samples_leaf = 2 : 0.7853810264385692
min_samples_leaf = 3 : 0.7931570762052877
min_samples_leaf = 4 : 0.7916018662519441
min_samples_leaf = 5 : 0.7869362363919129
min_samples_leaf = 6 : 0.7869362363919129
min_samples_leaf = 7 : 0.7884914463452566


Con el valor de max_depth ya seleccionado, continuamos probando valores para min_samples_leaf, en este caso el mejor resultado se obtiene con un valor de 3.

In [13]:
model = DecisionTreeClassifier(criterion='entropy', random_state=12345, max_depth=5, min_samples_leaf=3)
model.fit(features_train, target_train) 
predictions = model.predict(features_valid) 
result = accuracy_score(target_valid, predictions)
print(f'criterion = entropy : {result}')

criterion = entropy : 0.7962674961119751


Continuamos cambiando el criterion del modelo, el valor por defecto con el que hemos realizado las pruebas anteriores es "gini" por lo que procedimos a cambiarlo por "entropy" obteniendo una exactitud mayor con los hiperparámetros antes escogidos por lo que procederemos a quedarnos con este último criterio.

In [14]:
for split in range(2,8):
    model = DecisionTreeClassifier(criterion='entropy', random_state=12345, max_depth=5, min_samples_leaf=3, min_samples_split=split)
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    print(f' min_samples_split = {split} : {result}')

 min_samples_split = 2 : 0.7962674961119751
 min_samples_split = 3 : 0.7962674961119751
 min_samples_split = 4 : 0.7962674961119751
 min_samples_split = 5 : 0.7962674961119751
 min_samples_split = 6 : 0.7962674961119751
 min_samples_split = 7 : 0.7962674961119751


Finalmente modificaremos los valores de "min_samples_split", en este caso observamos que modificar este hiperparámetro no afecta la exactitud de nuestro modelo por lo que nos quedaremos con el valor por defecto de 2.

Hemos realizado pruebas en distintos hiperparámetros de nuestro algoritmo de aprendizaje "árbol de decisión" quedándonos finalmente con un "max_depth" de 5, "min_samples_leaf" de 3, "criterion" de "entropy" y el resto con sus valores por defecto.

### 6.2 Bosque aleatorio

In [15]:
for est in range(10, 101, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    print(f'n_estimators = {est} : {result}')

n_estimators = 10 : 0.7869362363919129
n_estimators = 20 : 0.7791601866251944
n_estimators = 30 : 0.7853810264385692
n_estimators = 40 : 0.7900466562986003
n_estimators = 50 : 0.7884914463452566
n_estimators = 60 : 0.7853810264385692
n_estimators = 70 : 0.7838258164852255
n_estimators = 80 : 0.7869362363919129
n_estimators = 90 : 0.7900466562986003
n_estimators = 100 : 0.7838258164852255


En el caso del algoritmo "bosque aleatorio" comenzamos probando distintos valores para "n_estimators", en este caso concluimos que la mayor exactitud la obtenemos con los valores de 40 y 90, utilizaremos el primero para reducir tiempos de carga.

In [16]:
model = RandomForestClassifier(random_state=12345, n_estimators=40, criterion='entropy')
model.fit(features_train, target_train) 
predictions = model.predict(features_valid) 
result = accuracy_score(target_valid, predictions)
print(f'criterion = entropy : {result}')

criterion = entropy : 0.7838258164852255


Igual que con el algoritmo anterior probamos con el valor de "entropy" para "criterion", en este caso se obtiene un mejor resultado con el valor por defecto "gini", por lo que mantendremos este último.

In [17]:
for split in range(2,8):
    model = RandomForestClassifier(random_state=12345, n_estimators=40, min_samples_split=split)
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    print(f'min_samples_split = {split} : {result}')

min_samples_split = 2 : 0.7900466562986003
min_samples_split = 3 : 0.7807153965785381
min_samples_split = 4 : 0.7838258164852255
min_samples_split = 5 : 0.7853810264385692
min_samples_split = 6 : 0.7869362363919129
min_samples_split = 7 : 0.7900466562986003


Al probar con diversos valores para "min_samples_split", en este caso tenemos los mejores resultados con el valor por defecto y con el valor de 7, por lo que dejaremos el primero por mayor simplicidad en el modelo.

In [18]:
for depth in range(1,10):
    model = RandomForestClassifier(random_state=12345, n_estimators=40, max_depth=depth)
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    print(f'min_samples_split = {depth} : {result}')

min_samples_split = 1 : 0.7573872472783826
min_samples_split = 2 : 0.7807153965785381
min_samples_split = 3 : 0.7838258164852255
min_samples_split = 4 : 0.7869362363919129
min_samples_split = 5 : 0.7916018662519441
min_samples_split = 6 : 0.7947122861586314
min_samples_split = 7 : 0.7900466562986003
min_samples_split = 8 : 0.7947122861586314
min_samples_split = 9 : 0.7931570762052877


Finalmente, se probó modificando el valor de "max_depth", obteniendo los mejores resultados con los valores de 6 y 8, escogiendo el valor de 6 para reducir los tiempos de cálculo.

Con las pruebas realizados con el algoritmo de "bosque aleatorio", finalmente nos quedamos con un valor de "n_estimators" de 40, un valor de "max_depth" de 8 y el resto de los hiperparámetros por defecto. 

### 6.3 Regresión logística

In [19]:
for solv in ['liblinear','newton-cg','lbfgs']:
    model = LogisticRegression(random_state=12345, solver=solv)
    model.fit(features_train, target_train) 
    predictions = model.predict(features_valid) 
    result = accuracy_score(target_valid, predictions)
    print(f'solver = {solv} : {result}')    

solver = liblinear : 0.6967340590979783
solver = newton-cg : 0.7589424572317263
solver = lbfgs : 0.7589424572317263


C:\Users\digom\anaconda3\lib\site-packages\scipy\optimize\_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\digom\anaconda3\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\digom\anaconda3\lib\site-packages\scipy\optimize\_linesearch.py:415: LineSearchWarning: Rounding errors prevent the line search from converging
  warn(msg, LineSearchWarning)
C:\Users\digom\anaconda3\lib\site-packages\scipy\optimize\_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\digom\anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


Como último algoritmo de aprendizaje estudiamos la "regresión logística", en este caso probamos distintos valores para "solver" concluyendo que los mejores resultados se obtienen con los valores de "newton-cg" y "lbfgs", elegiremos este último por ser el valor por defecto del algoritmo.

## 7. Calidad y elección del modelo

Finalmente procederemos a evaluar los tres algoritmos con los hiperparámetros previamente escogidos y haciendo uso de nuestro conjunto de prueba para elegir el que tenga la mayor exactitud posible.

In [20]:
model = DecisionTreeClassifier(criterion='entropy', random_state=12345, max_depth=5, min_samples_leaf=3)
model.fit(features_train, target_train) 
predictions = model.predict(features_test) 
result = accuracy_score(target_test, predictions)
print(f'Decision Tree accuracy : {result}')

Decision Tree accuracy : 0.7667185069984448


In [21]:
model = RandomForestClassifier(random_state=12345, n_estimators=40, max_depth=6)
model.fit(features_train, target_train) 
predictions = model.predict(features_test) 
result = accuracy_score(target_test, predictions)
print(f'Random Forest accuracy : {result}')

Random Forest accuracy : 0.7807153965785381


In [22]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train) 
predictions = model.predict(features_test) 
result = accuracy_score(target_test, predictions)
print(f'Logistic Regression accuracy : {result}')

Logistic Regression accuracy : 0.7262830482115086


Observando los resultados obtenidos con los distintos modelos, podemos descartar inmediatamente la regresión logística ya que no cumple el umbral de exactitud requerido de 0.75, entre los dos restantes el que mayor exactitud posee es el modelo creado con el algoritmo de aprendizaje de bosque aleatorio, llegando a un 0.78 de exactitud, por lo que será este el elegido para cumplir el fin del proyecto.

### 7.1 Prueba de cordura

Se procederá a realizar una prueba de cordura al modelo.

In [23]:
len(target_test)

643

In [24]:
sanity_test = [0]*643
result = accuracy_score(target_test,sanity_test)
result

0.6889580093312597

Al realizar una prueba de cordura con un modelo que predijera que todos los planes son 0 (Smart), la exactitud del modelo es de un 68,8%. Debido a lo anterior, podemos afirmar que nuestro modelo es más exacto que un modelo aleatorio. 

## 8. Conclusión general

Comenzamos el proyecto realizando una rápida exploración de la base de datos entregada por Megaline no encontrando ningún parámetro que hubiera que ser abordado, a su vez se pudieron identificar las columnas "características" y "objetivo" de nuestro modelo a realizar, además de que se dio constancia que la columna "objetivo" es de tipo categórica.

Continuamos realizando una segmentación de los datos en base a la identificación de las columnas realizadas en la sección anterior. Debido a que se nos entregó una sola base de datos y no poseíamos un conjunto de datos separados para realizar las pruebas finales procedimos a segmentar esta base en tres conjuntos, de entrenamiento, validación y prueba, en una proporción de 3:1:1.

Damos paso al entrenamiento de los modelos y mejoramiento de los mismos a través de probar distintos valores para sus hiperparámetros, con esto obtuvimos un set de valores definidos que nos otorgaban la mayor exactitud posible para cada modelo basado en un algoritmo de aprendizaje distinto.

Finalmente procedimos a comparar los distintos modelos con el conjunto de datos de prueba para poder analizar cuál es el más exacto, los resultados finales nos entregaron que el modelo que entregaba valores más exactos en el que está basado en el bosque aleatorio por lo cual sería este modelo el que le entregaríamos a Megaline para que pueda recomendar sus nuevos planes a sus clientes.